# Sistema de recomendación de chistes

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de chistes, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://goldberg.berkeley.edu/jester-data/

1- Cargue las bases de datos de calificaciones, usuarios y chistes. Puede usar para este objetivo la biblioteca pandas.

In [2]:
calificaciones = pd.read_csv("Datasets/rating.csv", sep=";", usecols=range(1,101))
usuarios       = pd.read_csv("Datasets/users.csv",  sep=";")
chistes        = pd.read_csv("Datasets/jokes.csv",  sep="\n")

In [3]:
def imprimir_usuario(indice):
    columnas = usuarios.columns
    usuario = usuarios.to_numpy()[indice]
    
    print("\x1b[1;33m" + "Usuario #{}".format(usuario[0]))
    
    for i in range(1, len(columnas)):
        print("\x1b[1;35m" + columnas[i].upper() + "\x1b[0m" +": {}".format(usuario[i]), end="\t")
        if i == 1:
            print(end="\t")
    
    print("\n")
    
def imprimir_chistes(chistes):
    if len(chistes):
        print("\x1b[1;32m" + "Chistes recomendados:\n" + "\x1b[0m")
    
        for i, chiste in enumerate(chistes):
            print("\x1b[1;36m" + str(1+i) + ". " + "\x1b[0m" + chiste)
            
        print("\x1b[1;34m" + "\nRecomendaciones totales: " + "\x1b[0m" + str(len(chistes)))
    else:
        print("\x1b[1;31m" + "No hay recomendaciones para este usuario" + "\x1b[0m")
        
    print("\n")

2- La matriz de calificaciones debe pasar por un proceso para poder se usada en el algoritmo de factorización de matrices. Para esto se debe:
    - Redondear los valores de las calificaciones y trabajar solo con valores enteros.
    - Cambiar las calificaciones 99 por 0.
    - Cambiar el rango de calificaciones de -10 a 10 por 0 a 10.
    
Recuerde que debe garantizar que la matriz de calificaciones deben ser números enteros.

In [4]:
calificaciones = np.round(calificaciones)
calificaciones = np.round((10 + calificaciones)/2)
calificaciones[calificaciones > 10] = 0

3- Usando el algoritmo de vecinos más cercanos de scikit-learn programe una función que reciba como parámetro las calificaciones de un usuario y la matrix de calificaciones. Usando 20 vecinos devuelva los indices de los vecinos más cercanos al usuario.

Link a scikit-learn: https://scikit-learn.org/stable/modules/neighbors.html

In [5]:
def indices_cercanos(matriz, fila, n=20):
    nbrs = NearestNeighbors(n_neighbors=n, algorithm="auto").fit(matriz)
    distancias, indices = nbrs.kneighbors(fila)
    return indices[0]

def interpretar_indices(matriz, indices):
    n = len(indices)
    R = list()
    
    for i in range(n):
        R.append(matriz[indices[i]])
    
    return np.array(R)

4- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas.

In [51]:
def factorizacion(R, P, Q, K, steps=7500, alpha=0.0002, beta=0.02, error=0.005):
    Q = Q.T
    err_relativo = 1
    err_anterior = 0
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - P[i,:] @ Q[:,j]
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
                   
        err_actual = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    err_actual = err_actual + pow(R[i][j] - P[i,:] @ Q[:,j], 2)
                    for k in range(K):
                        err_actual = err_actual + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
                        
        err_relativo = error_relativo(err_actual, err_anterior)
        err_anterior = err_actual
        
        if err_relativo < error:
            break
    return P, Q.T

def error_relativo(actual, anterior):
    return abs((actual - anterior) / actual)

5- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar lo siguiente:

- Generar R usando el algoritmo de vecinos más cercanos.
- Defina K.
- Defina P y Q.

In [52]:
def r(matriz, indice, n=20):
    fila = np.array([matriz[indice]])
    
    indices = indices_cercanos(matriz, fila, n)
    R = interpretar_indices(matriz, indices)
    
    return R

def r_prima(R, k):
    N = R.shape[0]
    M = R.shape[1]

    np.random.seed(0)
    P = np.random.rand(N,k)
    Q = np.random.rand(M,k)
    
    nP, nQ = factorizacion(R, P, Q, k, alpha=0.005)
    nR = nP @ nQ.T
    
    nR[nR > 10] = 10
    nR[nR <  0] = 0
    
    return np.round(nR)

6- Realice recomendaciones para los usuarios: 1, 470, 1241, 3044, 5758, 8105, 8899, 10597, 17391, 19821. Devueva como resultado todos los chistes que tiene una calificaciones de 7 o superior.

In [46]:
def sugerir(usuario, prediccion, cond):
    recomendaciones = list()
    
    for i in range(len(usuario)):
        if cond(prediccion[i]):
            recomendaciones.append(i)
                    
    return recomendaciones

def sugerir_otros(usuario, nR, cond):
    recomendaciones = list()
    
    for elemento in range(len(usuario)):
        if usuario[elemento] == 0:
            for vecino in range(len(nR)):
                if cond(nR[vecino][elemento]) and not elemento in recomendaciones:
                    recomendaciones.append(elemento)
                    
    return recomendaciones

def recomendar_chistes(R, condicion, k=10):
    nR = r_prima(R, k)
    
    recomendaciones = sugerir(R[0], nR[0], condicion)
    
    print(str(recomendaciones) + "\n")
    print(str(len(recomendaciones)) + "\n")
    
    chistes_recomendados = interpretar_indices(chistes.to_numpy().T[0], recomendaciones)
    
    return chistes_recomendados

def recomendar_chistes_por_calificacion(usuario, k=10):
    R = r(calificaciones.to_numpy(), usuario)
    
    condicion = lambda calificacion: calificacion >= 7
    
    recomendacion = recomendar_chistes(R, condicion, k)
    return recomendacion

In [53]:
k = 20
lista_usuarios = [1, 470, 1241, 3044, 5758, 8105, 8899, 10597, 17391, 19821]

for usuario in lista_usuarios:
    imprimir_usuario(usuario)
    recomendaciones = recomendar_chistes_por_calificacion(usuario, k)
    #chistes.to_numpy().T[0]imprimir_chistes(recomendaciones)

Usuario #2
AGE: 73		GENDER: female	COUNTRY: U.S	LANGUAJE: English	CATEGORY: Adventure	

[0, 2, 3, 8, 9, 10, 11, 12, 13, 14, 16, 21, 22, 23, 25, 26, 29, 30, 32, 33, 36, 39, 40, 42, 43, 49, 51, 56, 59, 63, 64, 66, 69, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 93]

50

Usuario #471
AGE: 52		GENDER: other	COUNTRY: Colombia	LANGUAJE: French	CATEGORY: Comedy	

[1, 3, 4, 5, 9, 11, 23, 26, 27, 28, 31, 33, 34, 35, 37, 42, 46, 48, 49, 52, 53, 54, 55, 57, 60, 61, 62, 64, 65, 67, 68, 69, 78, 84, 87, 88, 89, 90, 91]

39

Usuario #1242
AGE: 14		GENDER: female	COUNTRY: Japan	LANGUAJE: Spanish	CATEGORY: Animation	

[8, 12, 15, 17, 26, 28, 31, 33, 34, 35, 36, 38, 41, 45, 49, 55, 56, 57, 60, 67, 71, 78, 90, 91, 92, 94, 95, 96, 98, 99]

30

Usuario #3045
AGE: 31		GENDER: female	COUNTRY: Spain	LANGUAJE: Spanish	CATEGORY: Comedy	

[26, 27, 30, 31, 33, 34, 46, 47, 48, 49, 53, 55, 60, 61, 64, 65, 67, 68, 71, 72, 88, 98]

22

Usuario #5759
AGE: 46		GENDER: female	COUNTRY: Russia	LANGUAJE

7- Programe un algoritmo para realizar recomendaciones a usuarios con base en la informacion básica en el dataset usuarios. Use el algortimo de vecinos más cercanos y el algoritmo de factorización de matrices para este objetivo.

In [49]:
def procesar_tabla(tabla):
    matriz = tabla.to_numpy().T
    
    for columna in range(len(matriz)):
        tipos = list()
        
        for fila in range(len(matriz[columna])):
            dato = matriz[columna][fila]
            
            if isinstance(dato, str):
                if not dato in tipos:
                    tipos.append(dato)
                matriz[columna][fila] = tipos.index(dato)
            else:
                break
    
    return matriz.T

def recomendar_chistes_por_usuario(usuario, k=10):
    matriz_usuarios = procesar_tabla(usuarios)[:,1:]
    similares = indices_cercanos(matriz_usuarios, [matriz_usuarios[usuario]])
    R = interpretar_indices(calificaciones.to_numpy(), similares)
    
    condicion = lambda calificacion: calificacion >= 7
    
    recomendacion = recomendar_chistes(R, condicion, k)
    return recomendacion

In [50]:
k = 20
lista_usuarios = [1, 470, 1241, 3044, 5758, 8105, 8899, 10597, 17391, 19821]

for usuario in lista_usuarios:
    imprimir_usuario(usuario)
    recomendaciones = recomendar_chistes_por_usuario(usuario, k)
    #imprimir_chistes(recomendaciones)

Usuario #2
AGE: 73		GENDER: female	COUNTRY: U.S	LANGUAJE: English	CATEGORY: Adventure	

[0, 2, 5, 8, 10, 11, 12, 13, 20, 26, 27, 28, 30, 31, 33, 34, 35, 39, 40, 42, 45, 47, 48, 49, 51, 52, 53, 55, 59, 61, 64, 65, 67, 68, 71, 72, 73, 75, 76, 77, 78, 79, 80, 82, 83, 84, 87, 88, 89, 92, 93, 99]

52

Usuario #471
AGE: 52		GENDER: other	COUNTRY: Colombia	LANGUAJE: French	CATEGORY: Comedy	

[1, 2, 8, 9, 11, 20, 26, 27, 28, 30, 31, 34, 35, 42, 47, 48, 49, 52, 53, 54, 55, 61, 64, 65, 67, 68, 70, 71, 75, 77, 83, 86, 87, 88, 93, 95]

36

Usuario #1242
AGE: 14		GENDER: female	COUNTRY: Japan	LANGUAJE: Spanish	CATEGORY: Animation	

[2, 20, 26, 27, 28, 31, 33, 34, 47, 49, 52, 57, 61, 64, 65, 67, 68, 88, 90, 92, 93]

21

Usuario #3045
AGE: 31		GENDER: female	COUNTRY: Spain	LANGUAJE: Spanish	CATEGORY: Comedy	

[26, 31, 34, 47, 49, 53, 67, 88]

8

Usuario #5759
AGE: 46		GENDER: female	COUNTRY: Russia	LANGUAJE: English	CATEGORY: ScienceFiction	

[2, 8, 9, 10, 20, 28, 30, 31, 34, 39, 49, 52, 53, 61, 64, 